## Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from itertools import islice

import sentencepiece as sp

from sklearn.metrics.pairwise import cosine_similarity
import arabicstopwords.arabicstopwords as stp

import tkseem as tk
import gensim

In [ ]:
df = pd.read_csv('all_poems_tokenized.csv')
df.head()

In [3]:
morph_embedder = gensim.models.Word2Vec.load("models/morph_word2vec_model.model")
word_embedder = gensim.models.Word2Vec.load("models/word_word2vec_model.model")
sp_embedder = gensim.models.Word2Vec.load("models/sp_word2vec_model.model")
bert_embedder = gensim.models.Word2Vec.load("models/bert_word2vec_model.model")
bert_doc_embedder = gensim.models.Doc2Vec.load("models/bert_doc2vec_model.model")

## Sentence Embedding - Old

### morph_t

In [12]:
def morph_embed(tokens_poem):
    clear_poem = [token for token in tokens_poem if token != '<UNK>']
    embedded_tokens = np.array([morph_embedder.wv.__getitem__(token) for token in clear_poem if token in morph_embedder.wv.key_to_index])
    return np.mean(embedded_tokens, axis=0)

df['morph_embd'] = df['morph_t'].apply(morph_embed)
df['morph_embd']

0        [-0.24765442, 0.38770875, -0.102871165, 0.6061...
1        [-0.21053302, 0.40086082, -0.0674093, 0.553667...
2        [-0.2367812, 0.38922733, -0.02944215, 0.555683...
3        [-0.23369573, 0.38332504, -0.08212776, 0.54824...
4        [-0.20537826, 0.39523986, -0.051928695, 0.5820...
                               ...                        
58015    [-0.20806184, 0.38307905, -0.04547936, 0.57579...
58016    [-0.22086722, 0.38448393, -0.07153136, 0.57813...
58017    [-0.23908353, 0.39568943, -0.07490797, 0.57204...
58018    [-0.24266043, 0.38576385, -0.097762905, 0.5437...
58019    [-0.24392053, 0.3556195, -0.052376214, 0.53299...
Name: morph_embd, Length: 58020, dtype: object

### word_t

In [14]:
def word_embed(tokens_poem):
    clear_poem = [token for token in tokens_poem if token != '<UNK>']
    embedded_tokens = np.array([word_embedder.wv.__getitem__(token) for token in clear_poem if token in word_embedder.wv.key_to_index])
    return np.mean(embedded_tokens, axis=0)

df['word_embd'] = df['word_t'].apply(word_embed)
df['word_embd']

/home/ellzo/.local/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/ellzo/.local/lib/python3.11/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


0        [0.36752653, -0.17306797, 0.82105994, 1.336765...
1        [0.36973467, -0.17719436, 0.80202323, 1.401817...
2        [0.34428057, -0.23966013, 0.80465615, 1.332442...
3        [0.35446733, -0.19623774, 0.8237479, 1.3555199...
4        [0.35435063, -0.16567777, 0.78858435, 1.330945...
                               ...                        
58015    [0.32918015, -0.28842545, 0.873193, 1.3634624,...
58016    [0.34123966, -0.21129483, 0.763187, 1.3122137,...
58017    [0.3883289, -0.10817233, 0.78995746, 1.3408959...
58018    [0.37741277, -0.19153947, 0.7831429, 1.2971675...
58019    [0.30487433, -0.25957322, 0.84724337, 1.279440...
Name: word_embd, Length: 58020, dtype: object

In [15]:
df['word_embd'].isna().sum()

1

### sp_t

In [20]:
def word_embed(tokens_poem):
    clear_poem = [token for token in tokens_poem if token not in ['-0-', '-1-', '-2-', '-3-']]
    embedded_tokens = np.array([sp_embedder.wv.__getitem__(token) for token in clear_poem if token in sp_embedder.wv.key_to_index])
    return np.mean(embedded_tokens, axis=0)

df['sp_embd'] = df['sp_t'].apply(word_embed)
df['sp_embd']

0        [-0.1606714, -0.24332365, -0.1781964, 0.160212...
1        [-0.1436229, -0.22236936, -0.13860583, 0.18322...
2        [-0.14439291, -0.24374737, -0.11658855, 0.1895...
3        [-0.12386649, -0.25546777, -0.14277692, 0.1044...
4        [-0.10791435, -0.25342014, -0.135347, 0.245345...
                               ...                        
58015    [-0.17274301, -0.23464988, -0.17849502, 0.1313...
58016    [-0.20550907, -0.18462388, -0.11567006, 0.2114...
58017    [-0.15006873, -0.24745524, -0.14741597, 0.2391...
58018    [-0.23535645, -0.20253615, -0.13503188, 0.1809...
58019    [-0.14040759, -0.20414302, -0.20031135, 0.1980...
Name: sp_embd, Length: 58020, dtype: object

## Search - Old

In [21]:
df.head()

,poem_text,morph_t,word_t,sp_t,morph_embd,word_embd,sp_embd
0,عيناك غابتا نخيل ساعة السحر او شرفتان راح يناي...,"['عينا', '##ك', 'غابتا', 'نخيل', 'ساعة', 'ال',...","['عيناك', '<UNK>', 'نخيل', 'ساعة', 'السحر', 'ا...","['▁عيناك', '▁غابت', 'ا', '▁نخيل', '▁ساعة', '▁ا...","[-0.24765442, 0.38770875, -0.102871165, 0.6061...","[0.36752653, -0.17306797, 0.82105994, 1.336765...","[-0.1606714, -0.24332365, -0.1781964, 0.160212..."
1,انا لا ازال و في يدي قدحي ياليل اين تفرق الشر...,"['<UNK>', 'لا', '<UNK>', 'و', 'في', 'يدي', 'قد...","['انا', 'لا', 'ازال', 'و', 'في', 'يدي', '<UNK>...","['▁انا', '▁لا', '▁ازال', '▁و', '▁في', '▁يدي', ...","[-0.21053302, 0.40086082, -0.0674093, 0.553667...","[0.36973467, -0.17719436, 0.80202323, 1.401817...","[-0.1436229, -0.22236936, -0.13860583, 0.18322..."
2,علي مقلتيك ارتشفت النجوم وعانقت امالي الايبة ...,"['علي', 'مقلتي', '##ك', 'ارتشفت', 'النج', '##و...","['علي', 'مقلتيك', '<UNK>', 'النجوم', '<UNK>', ...","['▁علي', '▁مقلتيك', '▁ا', 'رتشف', 'ت', '▁النجو...","[-0.2367812, 0.38922733, -0.02944215, 0.555683...","[0.34428057, -0.23966013, 0.80465615, 1.332442...","[-0.14439291, -0.24374737, -0.11658855, 0.1895..."
3,اساطير من حشرجات الزمان نسيج اليد البالية رواه...,"['اساط', '##ير', 'من', 'حشرجات', 'ال', '##زمان...","['<UNK>', 'من', '<UNK>', 'الزمان', '<UNK>', 'ا...","['▁اساطير', '▁من', '▁حشر', 'جات', '▁الزمان', '...","[-0.23369573, 0.38332504, -0.08212776, 0.54824...","[0.35446733, -0.19623774, 0.8237479, 1.3555199...","[-0.12386649, -0.25546777, -0.14277692, 0.1044..."
4,والتف حولك ساعداي ومال جيدك في اشتهاء كالزهرة ...,"['والت', '##ف', 'حول', '##ك', 'ساعدا', '##ي', ...","['<UNK>', 'حولك', '<UNK>', 'ومال', '<UNK>', 'ف...","['▁والتف', '▁حولك', '▁ساعد', 'ا', 'ي', '▁ومال'...","[-0.20537826, 0.39523986, -0.051928695, 0.5820...","[0.35435063, -0.16567777, 0.78858435, 1.330945...","[-0.10791435, -0.25342014, -0.135347, 0.245345..."


In [61]:
def query_to_vec(query, column_used, embedder):
    token_query = query.split()
    clear_query = [token for token in token_query if token not in ['-0-', '-1-', '-2-', '-3-', '<UNK>']]
    embedded_tokens = np.array([embedder.wv.__getitem__(token) for token in clear_query if token in embedder.wv.key_to_index])
    if len(embedded_tokens > 0):
        return np.mean(embedded_tokens, axis=0)
    return [0 for i in range(len(df[column_used].dropna()[0]))]

def search(query, column_used, embedder):
    embedding_arrays = np.array(df[column_used].dropna().tolist())
    
    embed_query = query_to_vec(query, column_used, embedder)
    
    similarities = cosine_similarity(embedding_arrays, np.array(embed_query).reshape(1, -1))
    
    max_index = np.argmax(similarities)
    
    return df.at[max_index, 'poem_text']

In [62]:
query = 'جمال القمر'

print("Result using Morphological:")
print(search(query, 'morph_embd', morph_embedder))
print()

print("Result using Word:")
print(search(query, 'word_embd', word_embedder))
print()

print("Result using SentencePiece:")
print(search(query, 'sp_embd', sp_embedder))
print()

Result using Morphological:
راي المهلب او باس الايازيد

Result using Word:
مت من حبه وبغض ابيه

Result using SentencePiece:
الياسمين الدمشقي له اظافر بيضاء تثقب جدران الذاكرة 



In [63]:
query = 'سيف'

print("Result using Morphological:")
print(search(query, 'morph_embd', morph_embedder))
print()

print("Result using Word:")
print(search(query, 'word_embd', word_embedder))
print()

print("Result using SentencePiece:")
print(search(query, 'sp_embd', sp_embedder))
print()

Result using Morphological:
كقنوان النخيل المخصلف

Result using Word:
اي فردوس انسل منه النساء وسكبن السراب علي سبات السابلة نهرب ماء السماء في سواد المساء نقطر شمسا نحاسية علي شحوب الصحراء نشك الاصابع بماس العسيب اي نعاس يغالب صحو الصبايا نستمطر القلب اشواقا حية ورحيقا يفور نستمطر الوقت عمرا وصبرا جميل نستمطر الطرقات وطنا يبد الوحشة المشتركة اي رياح تخاطف الاشرعة نمازج الطوفان باطياف تطير ونءلف من كل زوجين اثنين مهرا لمهرة الهاربة اي قمر علقته شهرزاد علي ليل القاء قلنا اقترب قلنا عصافير تحترق قامة تورق قلنا زرقاء تقرا اشارات المحاق غابة سدر تشجر مكعبات الفراغ قلنا هيانا الاهلة لعيد الاكف لحناء هيانا هيانا عرسا لبلاد اخينا بين القمح والمستحيل الجرح بالملح وضانا وهبنا خميرة الروح لاجنة المطلق ونقضنا في الصباح غزل المساء اي حلم تبتدي منه المليحة 

Result using SentencePiece:
 المنسرح شنطف يا عوذة السموات والارض وشمس النهار والقمر



In [65]:
query = 'سيوف الوغى'

print("Result using Morphological:")
print(search(query, 'morph_embd', morph_embedder))
print()

print("Result using Word:")
print(search(query, 'word_embd', word_embedder))
print()

print("Result using SentencePiece:")
print(search(query, 'sp_embd', sp_embedder))
print()

Result using Morphological:
 رب اناس فرضوا فافترضوا فعرضوا فاعترضوا فقبضوافقبضوا فقبضوا فانقرضوا

Result using Word:
 تاوبني الداء الذي انا حاذره كما اعتاد مكمونا من اليله عاءره وتاوب داءي من يعف مشاشه عن الجار لا يشقي به من يعاشره ومن يمنع الناب السمينة همها اذا الخف امسي وهو جدب مصادره واهتضم الخال العزيز وانتحي عليه اذا ضل الطريق مناقره ولا اشتكي العفي ولا يخدموني اذا هر دون الحم والفرث جازره ولا اصطفي لحم السنام ذخيرة اذا عز ريح المسك باليل قاتره ولا يامن الاعداء مني قذيعة ولا اشتم الحي الذي انا شاعره ولا اطرق الجارات باليل قابعا قبوع القرنبي اخطاته محافره اذا كنت متبوعا قضيت وان اكن انا التباع المولي فاني مياسره اءدي اليه غير معط ظلامة واحدو اليه حقه لا اغادره وماء تبدي اهله من مخافة فراخ الحمام الوراق في الصيف حاضره وردت بعيس قد طلحن وفتية اذا حرك الناقوس باليل زاجره قطعنا لهن الحوضفابتل شطره لشرب غشاش وهو ظمان ساءره وهن سمام واضع حكماته مخوية اعجازه وكراكره وظل كظل المضرحي رفعته يطير اذا هنت له الريح طاءره لبيض الوجوه ادلجوا كل ليلهم ويمهم حتي استرقت ظهاءره فاضحوا نشاوي بهالفلا 

## Bert + Word2Vec Embedding

In [4]:
df_by_line = pd.read_csv('all_poems_bert_processed.csv')
df_by_line.head()

,poem_text,bert_t,doc_vecs
0,خليلي لا تستعجلا أن تزودا وأن تجمعا شملي وت...,"['خليل', 'تستعجل', 'تزود', 'تجمع', 'شملي', 'تن...",[-0.15892723 0.05683741 0.06184767 -0.065754...
1,فما لبث يوما بسابق مغنم ولا سرعتي يوما بساب...,"['لبث', 'يوم', 'سابق', 'مغنم', 'سرع', 'يوم', '...",[-0.17856994 -0.00725703 -0.0295063 0.071196...
2,وإن تنظراني اليوم أقض لبانة وتستوجبا منا عل...,"['تنظراني', 'يوم', 'أقض', 'بان', 'تستوجب', 'تح...",[ 0.13100246 -0.0729535 0.1925837 -0.047332...
3,لعمرك ما نفس بجد رشيدة تؤامرني سرا لأصرم مرثدا,"['عمر', 'جد', 'رشيد', 'تؤامرني', 'سر', 'أصرم',...",[ 0.01832668 -0.01884459 -0.00330791 0.020911...
4,وإن ظهرت منه قوارص جمة وأفرع في لومي مرارا ...,"['ظهر', 'قوارص', 'جم', 'أفرع', 'لومي', 'مرار',...",[ 0.17065507 0.06104766 -0.1744521 0.142696...


In [5]:
def bert_embed(tokens_poem):
    clear_poem = [token for token in tokens_poem if not (stp.is_stop(token) or ('+' in token))]
    embedded_tokens = np.array([bert_embedder.wv.__getitem__(token) for token in clear_poem if token in bert_embedder.wv.key_to_index])
    return np.mean(embedded_tokens, axis=0)

df_by_line['bert_embd'] = df_by_line['bert_t'].apply(bert_embed)
df_by_line['bert_embd']

/home/ellzo/.local/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/ellzo/.local/lib/python3.11/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


0          [-0.1121496, 0.11217325, 0.12640332, -0.045057...
1          [-0.1121496, 0.11217325, 0.12640332, -0.045057...
2          [-0.11214961, 0.112173244, 0.12640332, -0.0450...
3          [-0.1121496, 0.11217325, 0.12640332, -0.045057...
4          [-0.1121496, 0.11217325, 0.12640332, -0.045057...
                                 ...                        
1831722    [-0.11214961, 0.112173244, 0.12640332, -0.0450...
1831723    [-0.1121496, 0.11217325, 0.12640332, -0.045057...
1831724    [-0.1121496, 0.11217325, 0.12640332, -0.045057...
1831725    [-0.1121496, 0.11217325, 0.12640332, -0.045057...
1831726    [-0.11214961, 0.112173244, 0.12640332, -0.0450...
Name: bert_embd, Length: 1831727, dtype: object

## Bert + Doc2Vec

In [6]:
def doc_to_vecs(string_vec):
    result_list = string_vec.strip("[]").split()
    return [float(item) for item in result_list]

df_by_line['doc_vecs'] = df_by_line['doc_vecs'].apply(doc_to_vecs)
df_by_line['doc_vecs'][0]

[-0.15892723,
 0.05683741,
 0.06184767,
 -0.06575432,
 0.08093528,
 -0.0816489,
 -0.1633275,
 -0.06563673,
 -0.10536509,
 -0.02272989,
 -0.03389442,
 -0.07554101,
 0.25467494,
 0.1010648,
 0.05529438,
 -0.12479908,
 0.07520141,
 0.18715027,
 -0.01095252,
 0.08666662]

## Search - Doc2Vec

In [7]:
def query_to_docvec(query):
    token_query = query.split()
    return bert_doc_embedder.infer_vector(token_query)

def calc_similarity(doc_vector, query_vector):
    return cosine_similarity(np.array(doc_vector).reshape(1, -1), np.array(query_vector).reshape(1, -1))[0, 0]

def search_doc2vec(query, N=3):
    query_vec = query_to_docvec(query)
    similarities_series = df_by_line['doc_vecs'].apply(lambda doc_vec: calc_similarity(doc_vec, query_vec))
    max_indices = np.argpartition(similarities_series.values.tolist(), -N)[-N:]
    
    return df_by_line.loc[max_indices, 'poem_text']

In [8]:
query = 'جمال القمر'
print(search_doc2vec(query))

1706167       وصل أكمل صلاة وسلام    على محمد شفيع ذي الأنام
919901     ما أنت يا شيخنا الكبير كمن    أضحى كبير الجمال...
121460       حبذا حصنها الحصين لقد    كان جمالا لكل حصن حصين
Name: poem_text, dtype: object


In [18]:
query = 'ثلوج الجبال'
print(search_doc2vec(query))

124661     وحتى اعترى البهمى من الصيف نافض    كما نفضت خي...
1819486     كيف جو العبقريات وقد    شالت الأطواد فيه كالهباء
358064            كأن على الجو فضفاضة    مساميرها فضة أو ذهب
Name: poem_text, dtype: object


In [10]:
query = 'قتال الوغى'
print(search_doc2vec(query))

1807944    ولها ديار غير شرقي الحمى    شحطت وشطت عن لقاء ...
968554     طبعتم سيوفا لم يلق لنجادها    سوى منكب المجد ا...
770164       فسطاك موت للأعادي قاتل    ونداك للعافين غيث صيب
Name: poem_text, dtype: object


In [11]:
query = 'صوم رمضان'
print(search_doc2vec(query))

133654                  يصبح للناس منه وجه    كأنه وجه مردقش
1047478         ولامه كعينه المعتله    جزمهما بحذف حرف العله
1593061    وهو أيضا ثلثاه ربع لثلث    منه فاعجب والثلث لل...
Name: poem_text, dtype: object


In [12]:
query = 'طول السفر'
print(search_doc2vec(query))

247937     أيصلد بعد طول القدح زندي    ولم يصلد لمن رجاك زند
694601     تلك اليراعة من تلك اليراعة إن    تقصر السمر عن...
1558570                          وسأسائلكما    عن طول مدتكما
Name: poem_text, dtype: object


In [11]:
query = 'اشتياق الأهل'

print("Result using Bert:")
print(search_doc2vec(query))
print()

Result using Bert:
1407822    إني أحبك حبا ليس يبلغه    فهمي ولا ينتهي وصفي ...
1407824    إني أحبك حبا ليس يبلغه    فهمي ولا ينتهي وصفي ...
1374886    وبه اشتقت للعذيب وغيري    ذو اشتياق إلى النقا ...
Name: poem_text, dtype: object



In [12]:
query = 'هجو السلطان'

print("Result using Bert:")
print(search_doc2vec(query))
print()

Result using Bert:
114182    إني وإن قال أقوام مديحهم    فأحسنوه وما حابوا ...
683516                   من لست أرضى مدحه    فكيف أرضى هجوه
254856    أما لئن كثرت في مدحكم بدعي    لتكثرن غدا في شت...
Name: poem_text, dtype: object



In [13]:
query = 'جور الخليفة'

print("Result using Bert:")
print(search_doc2vec(query))
print()

Result using Bert:
1319236    ما كنت أحسب أن أيام الصبا    تمضي ولا أن الزما...
332900     فيا فؤادي كم وجد وكم حزن    ويا زماني ذا جور و...
850785     أضحى به الجور مطوي البساط فلم    ينشر ومنه بسا...
Name: poem_text, dtype: object



In [31]:
def query_to_docvec(query):
    token_query = query.split()
    return bert_doc_embedder.infer_vector(token_query)

def calc_similarity(doc_vector, query_vector):
    return cosine_similarity(doc_vector, query_vector)[0, 0]

def search_doc2vec_3(query, N=3):
    query_vec = np.array(query_to_docvec(query)).reshape(1, -1)
    similarities_series = df_by_line['doc_vecs'].apply(lambda doc_vec: calc_similarity(np.array(doc_vec).reshape(1, -1), query_vec))
    similarities_values = similarities_series.values.tolist()
    
    tri_way_similarity = [(similarities_values[i] + similarities_values[i-1] + similarities_values[i-2])/3 for i in range(len(similarities_values))]    
    max_indices = np.argpartition(tri_way_similarity, -N)[-N:]
    
    return df_by_line.loc[max_indices, 'poem_text']

In [33]:
query = 'صوم رمضان'

print("Result using Bert:")
print(search_doc2vec_3(query))
print()

Result using Bert:
1584718    بيت الرسالة بيتهم    جبريل فيه كان خادم
834916             وقيامي وقعودي    وسجودي والركوع
1294509                  ومني معبود ومني عابدي    
Name: poem_text, dtype: object



In [34]:
query = 'ثلوج الجبال'

print("Result using Bert:")
print(search_doc2vec_3(query))
print()

Result using Bert:
75018         كم ضيغم عفرته بعرينه    ولكم فتكت بأفتك الفتاك
1297689                            إذا تردى وشيه المصورا    
482517     كانت له الأرض أياما فصار لها    دهرا طويل اللي...
Name: poem_text, dtype: object



In [35]:
query = 'شجاعة في أرض القتال'

print("Result using Bert:")
print(search_doc2vec_3(query))
print()

Result using Bert:
1016097           على ذاته الرحمن صلى وسلما    
491121     وفي منامي انتباهي    وبانتباهي منامي
1020850          فكان في سؤاله    جوابه عما سأل
Name: poem_text, dtype: object



## Search - Word2Vec

In [13]:
def query_to_vec_bert(query):
    token_query = query.split()
    embedded_tokens = np.array([bert_embedder.wv.__getitem__(token) for token in token_query if token in bert_embedder.wv.key_to_index])
    if len(embedded_tokens > 0):
        return np.mean(embedded_tokens, axis=0)
    return [0 for i in range(len(['bert_embd'].dropna()[0]))]

def search_bert(query, N=3):
    embedding_arrays = np.array(df_by_line['bert_embd'].dropna().tolist())
    
    embed_query = query_to_vec_bert(query)
    
    similarities = cosine_similarity(embedding_arrays, np.array(embed_query).reshape(1, -1))
    
    max_indices = np.argpartition(similarities.flatten(), -N)[-N:]
    
    return df_by_line.loc[max_indices, 'poem_text']

In [14]:
query = 'صوم رمضان'
print(search_bert(query))

1788670    لا تلم الكافر في هذا الزمن الكافر    فالجوع اب...
1812380    حاشا كمالك من نقص تقدمه    والبدر يعرف بعد الن...
1788656                      يسيل على الغربات كعري الصبح    
Name: poem_text, dtype: object


In [15]:
query = 'ثلوج الجبال'
print(search_bert(query))

862073                              أينما مشيت    أينما مشيت
1788524    له وداد سقيم ما يصح لنا    كأنما طرفه أعداه با...
1812419    ماكانت الدنيا تضيق بطالب    لو أن واسع صدرها م...
Name: poem_text, dtype: object


In [16]:
query = 'شجاعة في أرض القتال'
print(search_bert(query))

1812419    ماكانت الدنيا تضيق بطالب    لو أن واسع صدرها م...
1785946                              وبعض الورود الحزينة    
1786685                      إن أتى الصبح فهي ناري ونوري    
Name: poem_text, dtype: object


In [17]:
query = 'طول السفر'
print(search_bert(query))

1773885    قد حررت فيه مباحث من مضى    وكلامهم أضحى بغير ...
1828119    فدع هملان الدمع في رزئه وما    عليه عيون المكر...
1812405    وأفاض منها للبرية أنعما    نطق الزمان بشكرها و...
Name: poem_text, dtype: object
